# Fantasy Premier League Moneyball

#### Optimisation constraints:
- My budget (start at £100 million but can go up and down based on player prices changing)
- Fill a team of 15 players consisting of 
        - 2 x GKP
        - 5 x DEF
        - 5 x MID
        - 3 X FWD
- 4 of these players will be subs and do not generate you gamepoints.
- The 'starting 11' Can be of any combination/formation, providing 1 goalkeeper, at least 3 defenders and at least 1 forward are selected at all times.
- Only 1 free transfer can be made each week, further transfers deduct 4 points from your total points


#### We want to Maximise:
- Total points per player
- Player score (points per million x fixture multiplier) x chance to play


#### Caps
- You are given a selection of boosts / advantages (called Caps) which you can play at any time, these are:
        - 2 x Wildcards: Make unlimited free transfers to your team in a gameweek
        - 1 x Free Hit: Same as above, but team reverts back to previous lineup after 1 week
        - Triple Captain: Your captain scores you triple points for the next gameweek
        - Bench Boost: The points from all your subs are included in your total score for the gameweek




In [ ]:
# The plan:

# link to google sheet:
# https://docs.google.com/spreadsheets/d/1z-DRmtGm3pW-xp951SmK0VTQidB_CxWMNmmciaQQc8M/edit#gid=581306190

# 1. Pull data from fantasy premier league site to create datasets for:
    # Club data - DONE
    # Player data - DONE
    # Fixture data - DONE - add in the fixture multipliers here
    # My team data - DONE - need to add player score once calculated
    # Previous season player data - NEED TO CREATE DF FROM CSV :
        #'C:\Users\andre\Documents\GitHub\murry_code\Fantasy Premier League\FPL_Players-2019-2020 season'
    
# 2. Create player score based on:
    # A - Points per million this season - DONE
    # B - Points per million last season (proportionate to games played by team so far this season, with a lower weighting)
    #   - Requires appearances to be calculated 

    # C - Blended Nxt 3 Gameweek Multiplier (average(next gw multiplier, avergage(next 3 gw multiplier))
    
    # D - Chances of playing next round multiplier (excludes injury & suspended) - DONE 
    
    # Player score = ((A + B) x C)) x D
    
# 3. Create Optimal team:    
    # Create best team of: 
        # Using 80-82 mil
        # 1 x GK
        # 3 x DEF
        # 4 x MID
        # 3 X FOR

    # Leaving enough for Subs: 
        # 18-20mil
        # 1 x GK
        # 2 x DEF
        # 1 X MID

# 4. Figure out a way to make transfer most efficiently to maximise my team score, 
    # Considering 1 free transfer, then 4 points per transfer after that       
    # calculate points predicted to score in next gameweek of lowest in active team 
    # vs 
    # position vs highest points predicted of player not in team
    
# Additional ideas
    # Pull data from promoted teams from the EFL championship in previous year so there is data on newly promoted teams/players
    # Add logic of points per appearance to get around lower points for injured or booked players
    # Possibly mins per appearance to infer a likliness to play
    # Add in a 'strongest fixture of the season' flag for players for when to play triple captain
    # Add in 'strongest fixture of next gameweek' flag for current capatin / vice captain
    # Add in 'chance to appear' based on how often they play when not injured 
    

### Creating variables for the api urls and logging in using personal details

In [238]:
#############
## imports ##
#############

import pandas as pd
from datetime import datetime as dt
import requests
import numpy as np
import time


###########################
## URLs an Login Details ##
###########################

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

# Code for my team picks
my_team_id = '3366741'

current_gameweek = 4

print('--- Current gameweek set to '+ str(current_gameweek)+' ---')

################
### API URLs ###
################

## login url to create a session using my login details ## 
login_url = 'https://users.premierleague.com/accounts/login/'

## api url for the main dataset on player stats 
main_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

## api for my team
team_url = 'https://fantasy.premierleague.com/api/my-team/3366741/'

## api for god knows what??
entry_url = 'https://fantasy.premierleague.com/api/entry/3366741/'

## api for fixtures
fix_url = 'https://fantasy.premierleague.com/api/fixtures/'

print('--- API URL Variables Created ---')

#########################################################
### create a login session for my team specific pages ###
#########################################################

login = ''
password = ''

session = requests.session()

## Create session with login details for urls which require login ##
## Add pasword and login email
payload = {'password': password,
           'login': login,
           'redirect_uri': 'https://fantasy.premierleague.com/a/login',
           'app': 'plfpl-web'}
session.post(login_url, data=payload)

print('--- Session created for '+login+ '---')

#########################
### Useful Functions: ###
#########################

# Export a dataframe to csv
# Pass the dataframe through first then optional filname and export path

def export_to_csv(dataframe, filename = 'FPL_data', path = 'C:\\Python CSV Output\\'):
    print('----------------')
    print('Start csv export')
    filename = 'FPL_' + filename + '-' + str(dt.now().strftime("%Y%m%d-%H%M%S"))
    print('- filename: '+ filename)
    path = path+filename+'.csv'
    print('- save location: ' + path)
    dataframe.to_csv(path, index = False, encoding="utf8")
    print('- successfully exported')
    print('-----------------------')

print("--- 'export_to_csv' function created ---")    


--- Current gameweek set to 4 ---
--- API URL Variables Created ---
--- Session created for andrew_c_morris@hotmail.co.uk---
--- 'export_to_csv' function created ---


### Creating data frames and csv outputs

In [195]:
## CLUB DATAFRAME AND CSV EXTRACT ##
### Creating a dataframe for the club data and saving as csv ###

############################
#### dataframe creation ####
############################

## Converting the data from the url to json then pulling out the teams data to a separate 'club' data frame
main_req = requests.get(main_url)
main_json = main_req.json()
main_json.keys()
club_df = pd.DataFrame(main_json['teams'])

print("--- 'club_df' dataframe created ---")    

# club_df.head()

--- 'club_df' dataframe created ---


In [196]:
### PREVIOUS SEASON PLAYER DATA ###
### Creating a dataframe for the player data and saving as csv ###
### Several uses, will contain the player score used to determine best team / transfers ### 

############################
#### Dataframe Creation ####
############################

# import_previous_season_path = r'C:\Users\andrew.morris\Documents\GitHub\murry_code\Fantasy Premier League\\'
import_previous_season_path = r'C:\Users\andre\Documents\GitHub\murry_code\Fantasy Premier League\\'
prev_season_filename = "FPL_Players_2019_2020_season_fudged.csv"
prev_season_player_df = pd.read_csv(import_previous_season_path+prev_season_filename) 
previous_season_games = 38

prev_season_player_df['points_per_gameweek'] = round(prev_season_player_df.total_points / previous_season_games,2)
print("--- 'prev_season_player_df' dataframe created ---")    
    
# print(prev_season_player_df.head())                                                                                      

--- 'prev_season_player_df' dataframe created ---


In [240]:
### PLAYER DATA ###
### Creating a dataframe for the player data and saving as csv ###
### Several uses, will contain the player score used to determine best team / transfers ### 

##############################################################
#### full_player_df and slim_player_df Dataframe Creation ####
##############################################################

full_player_df = pd.DataFrame(main_json['elements'])         # Player dataframe
position_types_df = pd.DataFrame(main_json['element_types']) # Position dataframe
events_df = pd.DataFrame(main_json['events'])                # Gameweek dataframe

# Updating and creating some of the fields in full player dataframe: 
    # Pulling through position and team name instead of 'id', and changing data types
full_player_df['position'] = full_player_df.element_type.map(position_types_df.set_index('id').singular_name)
full_player_df['position_abv'] = full_player_df.element_type.map(position_types_df.set_index('id').singular_name_short)
full_player_df['team'] = full_player_df.team.map(club_df.set_index('id').name)
full_player_df['team_abv'] = full_player_df.team.map(club_df.set_index('name').short_name)
full_player_df['team_and_position'] = + full_player_df['team_abv'] + '-' + full_player_df['position_abv']

    # Pulling through data related to previous season
full_player_df['prev_season_points'] = full_player_df.id.map(prev_season_player_df.set_index('id').total_points).fillna(0.0)
full_player_df['prev_season_points_per_gw'] = full_player_df.id.map(prev_season_player_df.set_index('id').points_per_gameweek).fillna(0.0)
full_player_df['prev_season_minutes'] = full_player_df.id.map(prev_season_player_df.set_index('id').minutes).fillna(0.0)

    # Converting selected_by_percent to float data type
full_player_df['selected_by_percent'] = full_player_df.selected_by_percent.astype(float)

    # Creating chance of playing next round multiplier
full_player_df['nxt_round_play_chance'] = full_player_df['chance_of_playing_next_round']/100.0
full_player_df['nxt_round_play_chance'] = full_player_df.nxt_round_play_chance.fillna(1.0)
full_player_df['this_round_play_chance'] = full_player_df['chance_of_playing_this_round']/100.0
full_player_df['this_round_play_chance'] = full_player_df.this_round_play_chance.fillna(1.0)

    # Creating custom points_per_mil
full_player_df['cost_in_mil'] = full_player_df['now_cost']/10
full_player_df['points_per_mil'] = round(full_player_df['total_points']/full_player_df['cost_in_mil'],2)
full_player_df['prev_season_points_per_mil'] = ((round(full_player_df['prev_season_points']/full_player_df['cost_in_mil'],2))/38)*(current_gameweek-1)
full_player_df['total_points_inc_prv_season'] = full_player_df.total_points + full_player_df.prev_season_points_per_gw

full_player_df['current_gameweek'] = current_gameweek

print("--- 'full_player_df' dataframe created ---")

# Creating a slimline version of the elements table with relevant data points required for players
slim_player_df = full_player_df[['first_name',
                                 'second_name',
                                 'web_name',   
                                 'id',   
                                 'position_abv',
                                 'team_abv',
                                 'team_and_position',
                                 'selected_by_percent', 
                                 'current_gameweek',
                                 'this_round_play_chance',   
                                 'nxt_round_play_chance',      
                                 'status',
                                 'cost_in_mil',
                                 'minutes',
                                 'transfers_in',
                                 'transfers_out',   
                                 'total_points',
                                 'total_points_inc_prv_season',
                                 'prev_season_minutes',
                                 'prev_season_points',
                                 'prev_season_points_per_gw',
                                 'prev_season_points_per_mil',
                                 'points_per_mil']]

print("--- 'slim_player_df' dataframe created ---")

# export_to_csv(slim_player_df, filename = 'FPL_slim_player_df')  
export_to_csv(full_player_df, filename = 'FPL_full_player_df')  

# slim_player_df.head(5)  

--- 'full_player_df' dataframe created ---
--- 'slim_player_df' dataframe created ---
----------------
Start csv export
- filename: FPL_FPL_full_player_df-20200927-115545
- save location: C:\Python CSV Output\FPL_FPL_full_player_df-20200927-115545.csv
- successfully exported
-----------------------


In [198]:
### FIXTURE DATA ###
### Creating a dataframe for the fixtures ###
### Primarily used to create a gameweek difficulty multiplier which is applied to the player score

#########################################
#### full fixture dataframe creation ####
#########################################

## Getting fixture data from the api url
fix_req = requests.get(fix_url)

## Converting that data to json then dataframe
fix_json = fix_req.json()
fix_df = pd.DataFrame(fix_json)

# Replacing null game weeks with 99 and converting data type to int, required for matches not yet scheduled
fix_df['event'] = fix_df.event.fillna(99.0).astype(int)

# Adding club names and abbreviations as new columns
fix_df['home_team_full']  = fix_df.team_h.map(club_df.set_index('id').name)
fix_df['home_team_short'] = fix_df.team_h.map(club_df.set_index('id').short_name)
fix_df['away_team_full']  = fix_df.team_a.map(club_df.set_index('id').name)
fix_df['away_team_short'] = fix_df.team_a.map(club_df.set_index('id').short_name)

# Adding Home/Away Strengths to fix_df then blending the home and away strengths, 
# As there are no crowds this year the home advantage might be underplayed
# Logic which utilises the home and away scores separately is saved under random at the end of the code

# Keep an eye on this to see if they flip around at any point
    # Home team strength scores
fix_df['h_team_str_overall_home'] = fix_df.team_h.map(club_df.set_index('id').strength_overall_away)
fix_df['h_team_str_att_home'] = fix_df.team_h.map(club_df.set_index('id').strength_attack_away)
fix_df['h_team_str_def_home'] = fix_df.team_h.map(club_df.set_index('id').strength_defence_away)
fix_df['h_team_str_overall_away'] = fix_df.team_h.map(club_df.set_index('id').strength_overall_home)
fix_df['h_team_str_att_away'] = fix_df.team_h.map(club_df.set_index('id').strength_attack_home)
fix_df['h_team_str_def_away'] = fix_df.team_h.map(club_df.set_index('id').strength_defence_home)

    # Blending home and away scores for the home team
fix_df['h_team_str_overall']  = fix_df[['h_team_str_overall_home','h_team_str_overall_away']].mean(axis=1)
fix_df['h_team_str_att'] = fix_df[['h_team_str_att_home','h_team_str_att_away']].mean(axis=1)
fix_df['h_team_str_def'] = fix_df[['h_team_str_def_home','h_team_str_def_away']].mean(axis=1)

    # Away team strength scores
fix_df['a_team_str_overall_away'] = fix_df.team_a.map(club_df.set_index('id').strength_overall_home)
fix_df['a_team_str_att_away'] = fix_df.team_a.map(club_df.set_index('id').strength_attack_home)
fix_df['a_team_str_def_away'] = fix_df.team_a.map(club_df.set_index('id').strength_defence_home)
fix_df['a_team_str_overall_home'] = fix_df.team_a.map(club_df.set_index('id').strength_overall_away)
fix_df['a_team_str_att_home'] = fix_df.team_a.map(club_df.set_index('id').strength_attack_away)
fix_df['a_team_str_def_home'] = fix_df.team_a.map(club_df.set_index('id').strength_defence_away)
    
    # Blending home and away scores for the away team
fix_df['a_team_str_overall']  = fix_df[['a_team_str_overall_home','a_team_str_overall_away']].mean(axis=1)
fix_df['a_team_str_att'] = fix_df[['a_team_str_att_home','a_team_str_att_away']].mean(axis=1)
fix_df['a_team_str_def'] = fix_df[['a_team_str_def_home','a_team_str_def_away']].mean(axis=1)


# Creating home/away position specific multipliers
    # Basic logic is:
        # For goalkeepers or defenders - home team defense score / away team attack score
        # For midfielders - home team overall score / away team overall score
        # For forwards - home team attack score / away team defense score
    
    # Home team multipliers
fix_df['home_GKP_DEF_multi'] = fix_df['h_team_str_def'] / fix_df['a_team_str_att']
fix_df['home_MID_multi'] = fix_df['h_team_str_overall'] / fix_df['a_team_str_overall']
fix_df['home_FWD_multi'] = fix_df['h_team_str_att'] / fix_df['a_team_str_def']

    # Away team multiplier
fix_df['away_GKP_DEF_multi'] = fix_df['a_team_str_def'] / fix_df['h_team_str_att']
fix_df['away_MID_multi'] = fix_df['a_team_str_overall'] / fix_df['h_team_str_overall']
fix_df['away_FWD_multi'] = fix_df['a_team_str_att'] / fix_df['h_team_str_def']

# Renaming 'event' to 'gameweek'
fix_df = fix_df.rename(columns={"event": "gameweek"})

print("--- 'fix_df' dataframe created ---")


#######################################################
### SLIM FIXTURES DATAFRAME AND GAMEWEEK MULTIPLIER ###
#######################################################


# Creating a slim version of the fixtures dataframe with a row per position, per team, per fixture  
# Firstly done by creating a dataframe for each position for home and away teams then concatenating all the dataframes

###################################
### Goalkeepers playing at home ###
###################################

GKP_home_fix_df = fix_df[['gameweek', 
                          'kickoff_time',
                          'team_h',
                          'home_team_short',
                          'away_team_short',
                          'team_h_difficulty',
                          'home_GKP_DEF_multi'
                         ]].copy()

GKP_home_fix_df['position_abv'],GKP_home_fix_df['home_or_away'] = ['GKP','home']

GKP_home_fix_df = GKP_home_fix_df.rename(columns={'team_h':'team_id',
                                                  'home_team_short':'team_abv',
                                                  'away_team_short':'opponent_abv',
                                                  'team_h_difficulty':'match_difficulty',
                                                  'home_GKP_DEF_multi':'match_multiplier'
                                                 })

#################################
### Defenders playing at home ###
#################################

DEF_home_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_h',
                          'home_team_short',
                          'away_team_short',
                          'team_h_difficulty',
                          'home_GKP_DEF_multi'
                         ]].copy()

DEF_home_fix_df['position_abv'],DEF_home_fix_df['home_or_away'] = ['DEF','home']

DEF_home_fix_df = DEF_home_fix_df.rename(columns={'team_h':'team_id',
                                                  'home_team_short':'team_abv',
                                                  'away_team_short':'opponent_abv',
                                                  'team_h_difficulty':'match_difficulty',
                                                  'home_GKP_DEF_multi':'match_multiplier'
                                                 })

###################################
### Midfielders playing at home ###
###################################

MID_home_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_h',
                          'home_team_short',
                          'away_team_short',
                          'team_h_difficulty',
                          'home_MID_multi',
                         ]].copy()

MID_home_fix_df['position_abv'],MID_home_fix_df['home_or_away'] = ['MID','home']

MID_home_fix_df = MID_home_fix_df.rename(columns={'team_h':'team_id',
                                                  'home_team_short':'team_abv',
                                                  'away_team_short':'opponent_abv',
                                                  'team_h_difficulty':'match_difficulty',
                                                  'home_MID_multi':'match_multiplier',
                                                 })

################################
### Forwards playing at home ###
################################

FWD_home_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_h',
                          'home_team_short',
                          'away_team_short',
                          'team_h_difficulty',
                          'home_FWD_multi'
                         ]].copy()

FWD_home_fix_df['position_abv'],FWD_home_fix_df['home_or_away'] = ['FWD','home']

FWD_home_fix_df = FWD_home_fix_df.rename(columns={'team_h':'team_id',
                                                  'home_team_short':'team_abv',
                                                  'away_team_short':'opponent_abv',
                                                  'team_h_difficulty':'match_difficulty',
                                                  'home_FWD_multi':'match_multiplier'
                                                 })

################################
### Goalkeepers playing away ###
################################

GKP_away_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_a',
                          'away_team_short',
                          'home_team_short',
                          'team_a_difficulty',
                          'away_GKP_DEF_multi'
                         ]].copy()

GKP_away_fix_df['position_abv'],GKP_away_fix_df['home_or_away'] = ['GKP','home']

GKP_away_fix_df = GKP_away_fix_df.rename(columns={'team_a':'team_id',
                                                  'away_team_short':'team_abv',
                                                  'home_team_short':'opponent_abv',
                                                  'team_a_difficulty':'match_difficulty',
                                                  'away_GKP_DEF_multi':'match_multiplier'
                                                 })

##############################
### Defenders playing away ###
##############################

DEF_away_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_a',
                          'away_team_short',
                          'home_team_short',
                          'team_a_difficulty',
                          'away_GKP_DEF_multi'
                         ]].copy()

DEF_away_fix_df['position_abv'],DEF_away_fix_df['home_or_away'] = ['DEF','home']

DEF_away_fix_df = DEF_away_fix_df.rename(columns={'team_a':'team_id',
                                                  'away_team_short':'team_abv',
                                                  'home_team_short':'opponent_abv',
                                                  'team_a_difficulty':'match_difficulty',
                                                  'away_GKP_DEF_multi':'match_multiplier'
                                                 })

################################
### Midfielders playing away ###
################################

MID_away_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_a',
                          'away_team_short',
                          'home_team_short',
                          'team_a_difficulty',
                          'away_MID_multi'
                         ]].copy()

MID_away_fix_df['position_abv'],MID_away_fix_df['home_or_away'] = ['MID','home']

MID_away_fix_df = MID_away_fix_df.rename(columns={'team_a':'team_id',
                                                  'away_team_short':'team_abv',
                                                  'home_team_short':'opponent_abv',
                                                  'team_a_difficulty':'match_difficulty',
                                                  'away_MID_multi':'match_multiplier'
                                                 })

#############################
### Forwards playing away ###
#############################

FWD_away_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_a',
                          'away_team_short',
                          'home_team_short',
                          'team_a_difficulty',
                          'away_FWD_multi'
                         ]].copy()

FWD_away_fix_df['position_abv'],FWD_away_fix_df['home_or_away'] = ['FWD','home']

FWD_away_fix_df = FWD_away_fix_df.rename(columns={'team_a':'team_id',
                                                  'away_team_short':'team_abv',
                                                  'home_team_short':'opponent_abv',
                                                  'team_a_difficulty':'match_difficulty',
                                                  'away_FWD_multi':'match_multiplier'
                                                 })

## Combining all the above dataframes

slim_fix_df = pd.concat([GKP_home_fix_df,
                         DEF_home_fix_df,
                         MID_home_fix_df,
                         FWD_home_fix_df,
                         GKP_away_fix_df,
                         DEF_away_fix_df,
                         MID_away_fix_df,
                         FWD_away_fix_df
                        ])

# Adding boolean fields for identifying current, historic, future and next 5 games weeks 
slim_fix_df['is_current_gameweek'   ] = slim_fix_df.gameweek == current_gameweek 
slim_fix_df['is_historic_gameweek'  ] = slim_fix_df.gameweek <  current_gameweek
slim_fix_df['is_future_gameweek'    ] = slim_fix_df.gameweek >  current_gameweek
slim_fix_df['is_in_next_3_gameweeks'] = slim_fix_df['gameweek'].apply(lambda a: a in range(current_gameweek, current_gameweek+3))
slim_fix_df['is_in_next_5_gameweeks'] = slim_fix_df['gameweek'].apply(lambda a: a in range(current_gameweek, current_gameweek+5))

# Concatenating team_abv and position_abv to use for mapping 
slim_fix_df['team_and_position'] = slim_fix_df.team_abv + '-' + slim_fix_df.position_abv

# Sorting by kickoff datetime 
slim_fix_df = slim_fix_df.sort_values(by=['kickoff_time']).reset_index(drop = True)

print("--- 'slim_fix_df' dataframe created ---")

# export_to_csv(slim_fix_df, filename = 'FPL_slim_fix_df')
# slim_fix_df.head()

--- 'fix_df' dataframe created ---
--- 'slim_fix_df' dataframe created ---


In [241]:
##############################################
### Creating Gameweek Multiplier Dataframe ###
##############################################

# ##########################################
# ### Next 5 Gameweek Average Multiplier ###
# ##########################################

# # Not using this at the moment

# Creating a dataframe of just the next 5 gameweeks with columns 
next_5_gameweeks_multi_df = slim_fix_df[slim_fix_df.is_in_next_5_gameweeks == True]

# Creating a Pivot table of next 5 gameweeks with 'team_and_position' & average 'match_multiplier' 
next_5_gameweeks_multi_pivot = next_5_gameweeks_multi_df.pivot_table(index=['team_and_position'],
                                                          values=['match_multiplier'], 
                                                          aggfunc=np.mean)                                                  
# Transforming the pivot table back into a dataframe
next_5_gameweeks_multi_df = pd.DataFrame(next_5_gameweeks_multi_pivot.to_records()).rename(columns={"match_multiplier": "nxt_5_gw_avg_multi"})

print("--- 'next_5_gameweeks_multi_df' dataframe created ---")
# print(next_5_gameweeks_multi_df.head())

##########################################
### Next 3 Gameweek Average Multiplier ###
##########################################

# Creating a dataframe of just the next 3 gameweeks with columns 
next_3_gameweeks_multi_df = slim_fix_df[slim_fix_df.is_in_next_3_gameweeks == True]

# Creating a Pivot table of next 5 gameweeks with 'team_and_position' & average 'match_multiplier' 
next_3_gameweeks_multi_pivot = next_3_gameweeks_multi_df.pivot_table(index=['team_and_position'],
                                                          values=['match_multiplier'], 
                                                          aggfunc=np.mean)                                                  
# Transforming the pivot table back into a dataframe
next_3_gameweeks_multi_df = pd.DataFrame(next_3_gameweeks_multi_pivot.to_records()).rename(columns={"match_multiplier": "nxt_3_gw_avg_multi"})

print("--- 'next_3_gameweeks_multi_df' dataframe created ---")
# print(next_3_gameweeks_multi_df.head())

####################################

# Creating a dataframe of just the next gameweek with columns 'team_and_position' & 'match_multiplier'
gameweek_multiplier_df = slim_fix_df[slim_fix_df.is_current_gameweek == True].reset_index(drop = True)
gameweek_multiplier_df = gameweek_multiplier_df[['team_and_position', 'match_multiplier']]

# Renaming match multipiler to 'current_gw_multiplier'
gameweek_multiplier_df = gameweek_multiplier_df.rename(columns={"match_multiplier": "gw_multi"})

# Pulling the 5 week average match multiplier into 'gameweek_multiplier_df' 
gameweek_multiplier_df['nxt_5_gw_multi'] = gameweek_multiplier_df.team_and_position.map(next_5_gameweeks_multi_df.set_index('team_and_position').nxt_5_gw_avg_multi)
gameweek_multiplier_df['nxt_3_gw_multi'] = gameweek_multiplier_df.team_and_position.map(next_3_gameweeks_multi_df.set_index('team_and_position').nxt_3_gw_avg_multi)

# Taking an average of of the current gameweek multiplier and the next 5 week match multiplier
# This way match strength will still be a little weighted towards the next match
# However the upcoming matches are also taken into account to avoid have to transfer players out of the team again in upcoming weeks
gameweek_multiplier_df['nxt_3_wk_blended_match_multi'] = gameweek_multiplier_df[['gw_multi','nxt_3_gw_multi']].mean(axis=1)
    
# Gameweek 5 multiplier not being used currently
gameweek_multiplier_df['nxt_5_wk_blended_match_multi'] = gameweek_multiplier_df[['gw_multi','nxt_5_gw_multi']].mean(axis=1)

print("--- 'gameweek_multiplier_df' dataframe created ---")
# gameweek_multiplier_df.head(10)

--- 'next_5_gameweeks_multi_df' dataframe created ---
--- 'next_3_gameweeks_multi_df' dataframe created ---
--- 'gameweek_multiplier_df' dataframe created ---


In [200]:
### Player History Data ###
### Data frame containing a row per player history for each previous gameweek ###
## Primarily use to determine the appearances and average gameweek multiplier for games played
## these will be used to determine points per gameweek and then used to calculate a predicted player score ###

#######################################
### Create Player History Dataframe ###
#######################################

start_time = time.time()

## Creating a function to make an empty dataframe from any dataframe you pass through the function

def create_empty_df(dataframe_name):
    list_of_col = []
    for col in dataframe_name.columns: 
        list_of_col.append(col) 
    return pd.DataFrame(columns=list_of_col)


## Each player has an api url with their own history
## This function will iterate through all the players
## By inserting their element (id) into the url and appending to a blank dataframe

def create_player_history_table():
    # creating a blank dataframe from the first players url
    url = 'https://fantasy.premierleague.com/api/element-summary/1/'
    req = requests.get(url)
    json = req.json()
    df = pd.DataFrame(json['history'])
    player_history_df = create_empty_df(df)
    # creating a range containing each player id
    id_range = range(1,len(full_player_df)+1)
    
    for i in id_range:    
        i_url = 'https://fantasy.premierleague.com/api/element-summary/'+str(i)+'/'
        i_req = requests.get(i_url)
        i_json = i_req.json()
        i_df = pd.DataFrame(i_json['history'])
        player_history_df = player_history_df.append(i_df, ignore_index=True)
    return player_history_df

## Creating player_history df
player_history_df = create_player_history_table()

print("--- 'player_history_df' dataframe created ---")
print("--- Time taken to run: %s seconds ---" % (time.time() - start_time))
# player_history_df.head(50)

--- 'player_history_df' dataframe created ---
--- Time taken to run: 70.14052820205688 seconds ---


In [220]:
####################################################################
###        Creating a slimline version of player history         ###
### and combining with gameweek multipliers and full player data ###
####################################################################

start_time = time.time()

### Used to provide a score per match, adjusted based on that match difficulty

# Creating a slimline version of the player history dataframe
slim_player_history_df = player_history_df[['element', 'opponent_team', 'total_points', 'round', 'minutes']].copy()

slim_player_history_df['opponent_abv'] = slim_player_history_df.opponent_team.map((club_df.set_index('id').short_name))
# Renaming the 'round' field to 'gameweek'
slim_player_history_df = slim_player_history_df.rename(columns={'round':'gameweek', 'total_points': 'match_points'})

# merging with full player df to pull through required fields
slim_player_history_df = slim_player_history_df.merge(full_player_df[['id',
                                                                      'first_name',
                                                                      'second_name',
                                                                      'web_name',
                                                                      'position_abv',
                                                                      'team_abv']], 
                                                       how = 'left', left_on = ['element'], right_on = ['id']).copy()

# Combining the team and position abbreviations to create a lookup field for the fixture dataframe
slim_player_history_df['team_and_position'] = slim_player_history_df.team_abv + '-' + slim_player_history_df.position_abv

slim_player_history_df = slim_player_history_df.merge(slim_fix_df[['gameweek', 'team_and_position', 'match_multiplier']], 
                                                      how = 'left', left_on = ['gameweek', 'team_and_position'], right_on = ['gameweek', 'team_and_position']).copy()

slim_player_history_df['appearance'] = slim_player_history_df.minutes.apply(lambda x: 1 if x > 0 else 0)

# Assigning an appearance factor based on the number of mins they played. 1 if over 60 mins, otherwise the mins played / 60, 
slim_player_history_df['appearance_factor'] = slim_player_history_df.minutes.apply(lambda x: 1.0 if x > 60 else (x / 90))

# Adjusting the match points based on match difficulty, so tougher matches have points increased and easier matches have points decreased
slim_player_history_df['adjusted_match_points'] = slim_player_history_df['match_points'] / (slim_player_history_df['match_multiplier'] / 1)
slim_player_history_df['adjusted_match_points'] = slim_player_history_df['adjusted_match_points'].astype(float)

## Removing records where they did not play
slim_player_history_df = slim_player_history_df[slim_player_history_df.appearance == 1]

slim_player_history_df = slim_player_history_df[['id',
                                                 'first_name',
                                                 'second_name',
                                                 'web_name',
                                                 'team_abv',
                                                 'position_abv',
                                                 'team_and_position',
                                                 'opponent_abv',
                                                 'match_points',                                 
                                                 'match_multiplier',
                                                 'adjusted_match_points',
                                                 'appearance',
                                                 'appearance_factor']].copy()


print("--- 'slim_player_history_df' dataframe created ---")

# create average points per appearance pivot
avg_points_per_app_df = slim_player_history_df[['id','adjusted_match_points']].copy()
avg_points_per_app_df['adjusted_match_points'] = avg_points_per_app_df['adjusted_match_points'].astype(float)
avg_points_per_app_pivot = avg_points_per_app_df.pivot_table(index=['id'], 
                                                             values=['adjusted_match_points'],
                                                             aggfunc='mean')

# Then converting to a dataframe and renaming 
avg_points_per_app_df = pd.DataFrame(avg_points_per_app_pivot.to_records()).rename(columns={"adjusted_match_points": "avg_points_per_app"})

print("--- 'avg_points_per_app_df' dataframe created ---")

# slim_player_history_df.head()
avg_points_per_app_df.head()

--- 'slim_player_history_df' dataframe created ---
--- 'avg_points_per_app_df' dataframe created ---


,id,avg_points_per_app
0,3,0.897119
1,4,5.093496
2,6,6.297071
3,8,3.835391
4,9,2.119919


### Adding gameweek Multipliers To Players and Calculating Player Score

#### Version 1 of the player score calc:
   - Use total points per million 
   - Add on prev season total points per million (which has been divided by 38 and times by current game week number -1 in order to match the amount of gameweeks the current total points reflects)
   - Next 3 gameweek blended multiplier is then applied (weighted toward next gameweek)
   - Multiplied by change to play next round
   - Then any players with a 'Next 3 gameweek blended multiplier' of less than 1 is removed. 
    
    

#### Version 2 of the player score calc:



In [269]:
#############################################################
### Updating slim_player_df with the gameweek multipliers ###
#############################################################

# Adding current game week multipliers from slim_fix_df to slim_player_df 

new_slim_player_df = slim_player_df.copy()

# merging with full player df to pull through required fields
new_slim_player_df = new_slim_player_df.merge(gameweek_multiplier_df[['team_and_position',
                                                              'gw_multi',
                                                              'nxt_3_gw_multi',
                                                              'nxt_3_wk_blended_match_multi',
                                                              'nxt_5_gw_multi',
                                                              'nxt_5_wk_blended_match_multi']], 
                                                       how = 'left', on = ['team_and_position']).copy()

# new_slim_player_df['negative_3wk_multiplier'] = new_slim_player_df['nxt_3_wk_blended_match_multi'] < 1

# Using the players average points per appearance calculate the predicted score for:
    # next gameweek using next gameweek multiplier
    # average for next 3 gameweeks using the average of the next 3 gameweeks
    # average for next 5 gameweeks using the average of the next 5 gameweeks
new_slim_player_df['avg_points_per_app'] = new_slim_player_df.id.map(avg_points_per_app_df.set_index('id').avg_points_per_app)
new_slim_player_df['nxt_gw_pred_score']  = new_slim_player_df['avg_points_per_app'] * new_slim_player_df['gw_multi']
new_slim_player_df['nxt_3gw_pred_score'] = (new_slim_player_df['avg_points_per_app'] * new_slim_player_df['nxt_3_gw_multi'])
new_slim_player_df['nxt_5gw_pred_score'] = (new_slim_player_df['avg_points_per_app'] * new_slim_player_df['nxt_5_gw_multi'])
new_slim_player_df['nxt_gw_pred_score_per_mil'] =  new_slim_player_df['nxt_gw_pred_score']  / new_slim_player_df['cost_in_mil']
new_slim_player_df['nxt_3gw_pred_score_per_mil'] = new_slim_player_df['nxt_3gw_pred_score'] / new_slim_player_df['cost_in_mil']
new_slim_player_df['nxt_5gw_pred_score_per_mil'] = new_slim_player_df['nxt_5gw_pred_score'] / new_slim_player_df['cost_in_mil']

##################################
###### Player Score V1 Calc ######
##################################

# Player score v1 calc = 
    # if E = True then 0.0 else ((A+B) X C) X D
        # A - Current season points per mil (Float)
        # B - Previous season points per mil (Float)
        # C - Next 3 Gameweek Multiplier (Float)
        # D - Chance to play next round (Float)
        # E - Has Negative Multiplier (Boolean)

# create a variable containing the player score v1 calculation        
calc_player_score_v1 = (((new_slim_player_df.points_per_mil +
                       new_slim_player_df.prev_season_points_per_mil) *
                      new_slim_player_df.nxt_3_wk_blended_match_multi) *
                     new_slim_player_df.nxt_round_play_chance) * new_slim_player_df['nxt_3_wk_blended_match_multi'].apply(lambda x: 0.0 if x < 1 else 1.0)

new_slim_player_df['player_score_v1'] = calc_player_score_v1

##################################
###### Player Score V2 Calc ######
##################################

# Player score v2 calc = 
    # if C True then 0.0 else (A * B)
        # A = predicted player score for the next 3 gameweeks as points per million:
        #     (average points per appearance x average gw multiplier for next 3 weeks) / cost in million
        # B = Chance to play next round (Float)
        # C = Has Negative 3 GW Multiplier (their average multiplier for the next 3 weeks is less than 1)

calc_player_score_v2 = (new_slim_player_df.nxt_3gw_pred_score_per_mil * new_slim_player_df.nxt_round_play_chance) * new_slim_player_df['nxt_3_wk_blended_match_multi'].apply(lambda x: 0.0 if x < 1 else 1.0)

new_slim_player_df['player_score_v2'] = calc_player_score_v2

########################################
### End of player score calculations ###
########################################

new_slim_player_df = new_slim_player_df.sort_values(by=['player_score_v1'], ascending=False).reset_index(drop = True)

print("--- 'new_slim_player_df' updated ---")

new_slim_player_df.head(10)

--- 'new_slim_player_df' updated ---


,first_name,second_name,web_name,id,position_abv,team_abv,team_and_position,selected_by_percent,current_gameweek,this_round_play_chance,nxt_round_play_chance,status,cost_in_mil,minutes,transfers_in,transfers_out,total_points,total_points_inc_prv_season,prev_season_minutes,prev_season_points,prev_season_points_per_gw,prev_season_points_per_mil,points_per_mil,gw_multi,nxt_3_gw_multi,nxt_3_wk_blended_match_multi,nxt_5_gw_multi,nxt_5_wk_blended_match_multi,avg_points_per_app,nxt_gw_pred_score,nxt_3gw_pred_score,nxt_5gw_pred_score,nxt_gw_pred_score_per_mil,nxt_3gw_pred_score_per_mil,nxt_5gw_pred_score_per_mil,player_score_v1,player_score_v2
0,Dominic,Calvert-Lewin,Calvert-Lewin,164,FWD,EVE,EVE-FWD,29.7,4,1.0,1.0,a,7.4,248,1515806,171573,31,34.32,2621.0,126.0,3.32,1.344474,4.19,1.075472,0.979703,1.027587,0.989136,1.032304,10.068713,10.828616,9.864346,9.959327,1.463327,1.333020,1.345855,5.687154,1.333020
1,Romain,Saïss,Saïss,461,DEF,WOL,WOL-DEF,11.8,4,1.0,1.0,a,5.1,180,619933,140477,16,18.71,2703.0,103.0,2.71,1.594737,3.14,1.161765,1.119129,1.140447,1.074550,1.118157,8.267932,9.605392,9.252879,8.884307,1.883410,1.814290,1.742021,5.399715,1.814290
2,Virgil,van Dijk,van Dijk,250,DEF,LIV,LIV-DEF,29.1,4,1.0,1.0,a,6.5,180,255707,239284,14,18.68,3420.0,178.0,4.68,2.161579,2.15,1.263158,1.167673,1.215416,1.146667,1.204912,5.992424,7.569378,6.997194,6.871314,1.164520,1.076491,1.057125,5.240360,1.076491
3,Alisson,Ramses Becker,Alisson,252,GKP,LIV,LIV-GKP,13.3,4,1.0,1.0,a,6.0,180,129505,162442,15,18.21,2543.0,122.0,3.21,1.605000,2.50,1.263158,1.167673,1.215416,1.146667,1.204912,6.789773,8.576555,7.928236,7.785607,1.429426,1.321373,1.297601,4.989281,1.321373
4,Lucas,Digne,Digne,155,DEF,EVE,EVE-DEF,14.3,4,1.0,1.0,a,6.1,270,498358,169240,19,22.16,3031.0,120.0,3.16,1.552895,3.11,1.044843,1.000029,1.022436,0.996535,1.020689,6.260372,6.541106,6.260555,6.238678,1.072312,1.026321,1.022734,4.767512,1.026321
5,Timothy,Castagne,Castagne,498,DEF,LEI,LEI-DEF,10.7,4,1.0,1.0,a,5.6,180,599104,124726,23,23.00,0.0,0.0,0.00,0.000000,4.11,1.133028,1.116106,1.124567,1.102980,1.118004,10.155870,11.506881,11.335030,11.201727,2.054800,2.024112,2.000308,4.621970,2.024112
6,Michael,Keane,Keane,156,DEF,EVE,EVE-DEF,4.1,4,1.0,1.0,a,5.0,270,177367,48903,15,17.37,2556.0,90.0,2.37,1.421053,3.00,1.044843,1.000029,1.022436,0.996535,1.020689,4.721030,4.932735,4.721168,4.704670,0.986547,0.944234,0.940934,4.520244,0.944234
7,Richarlison,de Andrade,Richarlison,166,FWD,EVE,EVE-FWD,11.5,4,1.0,1.0,a,8.0,269,275842,194940,22,26.34,3070.0,165.0,4.34,1.627895,2.75,1.075472,0.979703,1.027587,0.989136,1.032304,6.923977,7.446541,6.783438,6.848754,0.930818,0.847930,0.856094,4.498669,0.847930
8,James,Justin,Justin,239,DEF,LEI,LEI-DEF,21.3,4,1.0,1.0,a,4.7,180,767252,171747,16,16.92,1010.0,35.0,0.92,0.588158,3.40,1.133028,1.116106,1.124567,1.102980,1.118004,7.111336,8.057339,7.937006,7.843665,1.714328,1.688725,1.668865,4.484950,1.688725
9,Reece,James,James,123,DEF,CHE,CHE-DEF,16.1,4,1.0,1.0,a,5.1,270,756492,165595,16,17.61,1510.0,61.0,1.61,0.944211,3.14,1.096916,1.073589,1.085253,1.058378,1.077647,4.803213,5.268722,5.156675,5.083616,1.033083,1.011113,0.996787,4.432400,1.011113


Table of Top Players Ordered By Total Points:


,cost_in_mil,team_abv,position_abv,web_name,id,avg_points_per_app,gw_multi,nxt_gw_pred_score,nxt_3_gw_multi,nxt_3gw_pred_score,player_score_v2
0,11.6,MCI,MID,De Bruyne,272,11.680451,1.285024,15.009662,1.197377,13.985905,1.205681
1,9.1,TOT,MID,Son,390,11.707317,0.984000,11.520000,1.063959,12.456103,1.368803
2,5.6,LEI,DEF,Castagne,498,10.155870,1.133028,11.506881,1.116106,11.335030,2.024112
3,12.1,LIV,MID,Salah,254,9.134831,1.253521,11.450704,1.180576,10.784365,0.891270
4,7.4,EVE,FWD,Calvert-Lewin,164,10.068713,1.075472,10.828616,0.979703,9.864346,1.333020


In [347]:
##########################################################
### TOP POINTS PLAYERS AND TOP PLAYER SCORE DATAFRAMES ###
###           BASED ON PLAYER SCORE CALC V2            ###
##########################################################


######################################################################
# Creating a data frame of players ordered by their predicted points #
######################################################################

points_top_players = new_slim_player_df[['cost_in_mil',
                                         'team_abv',
                                         'position_abv',
                                         'web_name',
                                         'id',
                                         'avg_points_per_app',
                                         'gw_multi',
                                         'nxt_gw_pred_score',
                                         'nxt_3_gw_multi',
                                         'nxt_3gw_pred_score',
                                         'player_score_v2'
                                        ]].copy()

points_top_players = points_top_players[points_top_players['player_score_v2'] > 0.0]
# points_top_players = points_top_players[points_top_players['nxt_3_gw_multi'] > 0.98]
points_top_players = points_top_players.sort_values(by=['nxt_3gw_pred_score'], ascending=False).reset_index(drop = True)


###########################################################################################################
# Creating a data frame of players ordered by their player score (to get the most cost effective players) #
###########################################################################################################

player_score_top_players = new_slim_player_df[['cost_in_mil',
                                               'team_abv',
                                               'position_abv',
                                               'web_name',
                                               'id',
                                               'avg_points_per_app',
                                               'gw_multi',
                                               'nxt_gw_pred_score',
                                               'nxt_3_gw_multi',
                                               'nxt_3gw_pred_score',
                                               'player_score_v2'
                                              ]].copy()

player_score_top_players = player_score_top_players[player_score_top_players['player_score_v2'] > 0.0]
# player_score_top_players = player_score_top_players[player_score_top_players['nxt_3_gw_multi'] > 0.98]
player_score_top_players = player_score_top_players.sort_values(by=['player_score_v2'], ascending=False).reset_index(drop = True)


print('Table of Top Players Ordered By Total Points:')
points_top_players.head(5)

# print('Table of Top Players Ordered By Player Score:')
# player_score_top_players.head(5)

################################################
### Optimal team logic Using Player Score V2 ###
################################################

# Function which produces the optimal team based on:
    # A specified number of star players (picking solely based on top points)
    # Filling the rest of the team based on points per million 
    # A set the budget you want to spend on your team
    # Specify the minimum cost you want left to spend on the final players (most likely your subs)

def get_optimal_team_objects(points_top_players=points_top_players, 
                           player_score_top_players=player_score_top_players,
                           budget = 100, 
                           star_player_limit = 3, 
                           gk = 2, 
                           df = 5, 
                           md = 5, 
                           fwd = 3, 
                           min_player_cost = 4.5):
    
    optimal_team = pd.DataFrame(columns = ['id', 
                                           'web_name', 
                                           'position_abv', 
                                           'cost_in_mil', 
                                           'nxt_3_gw_multi', 
                                           'nxt_3gw_pred_score', 
                                           'team_abv'])
    star_player_limit = star_player_limit
    budget = budget
    positions = {'GKP': gk, 'DEF': df, 'MID': md, 'FWD': fwd}
    club_player_count = dict((club, 3) for club in list(club_df['short_name'].unique()))
    players_left = 15
    min_player_cost = min_player_cost
    
    for index, player in points_top_players.iterrows():
        if len(optimal_team) < star_player_limit and budget >= player.cost_in_mil and positions[player.position_abv] > 0  and club_player_count[player.team_abv] > 0 and (budget - player.cost_in_mil) >= ((players_left - 1) * min_player_cost):
            my_dict = {'id': player.id,
                       'web_name': player.web_name, 
                       'position_abv': player.position_abv, 
                       'cost_in_mil': player.cost_in_mil,
                       'nxt_3_gw_multi': player.nxt_3_gw_multi, 
                       'nxt_3gw_pred_score': player.nxt_3gw_pred_score, 
                       'team_abv': player.team_abv}
            optimal_team = optimal_team.append(my_dict, ignore_index=True)
            budget -= player.cost_in_mil
            players_left = players_left - 1 
            positions[player.position_abv] = positions[player.position_abv] -1
            club_player_count[player.team_abv] = club_player_count[player.team_abv] -1
    for index, player in player_score_top_players.iterrows():
        if player.id not in optimal_team['id'].to_list() and budget >= player.cost_in_mil and positions[player.position_abv] > 0 and club_player_count[player.team_abv] > 0 and (budget - player.cost_in_mil) >= ((players_left - 1) * min_player_cost):
            my_dict2 = {'id':player.id,
                        'web_name': player.web_name, 
                        'position_abv': player.position_abv, 
                        'cost_in_mil': player.cost_in_mil, 
                        'nxt_3_gw_multi': player.nxt_3_gw_multi, 
                        'nxt_3gw_pred_score': player.nxt_3gw_pred_score, 
                        'team_abv': player.team_abv}
            optimal_team = optimal_team.append(my_dict2, ignore_index=True) 
            budget -= player.cost_in_mil
            players_left = players_left - 1 
            positions[player.position_abv] = positions[player.position_abv] -1
            club_player_count[player.team_abv] = club_player_count[player.team_abv] -1
    return optimal_team

# functions to evaluate optimal team 
def valid_team(team):
    if (len(team)) < 15:
        return 'Valid Team: False - Cannot make selection of 15 players with current star_player_limit, budget & min_player_cost'
    else:
        return 'Valid Team: True'
    
def terrible_subs(team):
    if team.nxt_3gw_pred_score.min() < 3:
        return 'Terrible Subs! Some players have less than 3 points on average over next 3 gameweeks'
    else:
        return 'Great Team! All players expected to score more than 3 points on average over next 3 gameweeks'

    
# Run optimal team function to create optimal_team_dataframe with custom parameters    
optimal_team_df = get_money_team_objects(star_player_limit = 3, 
                                         budget = 99, 
                                         min_player_cost = 5)  

print('Total Team Points: ' + str(round(optimal_team_df['nxt_3gw_pred_score'].sum(),2)))
print('Total Team Cost: ' + str(round(optimal_team_df['cost_in_mil'].sum(),2)))
print('Total Team Multiplier: ' + str(round(optimal_team_df['nxt_3_gw_multi'].sum(),2)))
print(valid_team(optimal_team_df))
print('--------------------------')
print(terrible_subs(optimal_team_df))

optimal_team_df.head(15)

Table of Top Players Ordered By Total Points:
Total Team Points: 123.39
Total Team Cost: 97.1
Total Team Multiplier: 16.43
Valid Team: True
--------------------------
Great Team! All players expected to score more than 3 points on average over next 3 gameweeks


,id,web_name,position_abv,cost_in_mil,nxt_3_gw_multi,nxt_3gw_pred_score,team_abv
0,272,De Bruyne,MID,11.6,1.197377,13.985905,MCI
1,390,Son,MID,9.1,1.063959,12.456103,TOT
2,498,Castagne,DEF,5.6,1.116106,11.335030,LEI
3,461,Saïss,DEF,5.1,1.119129,9.252879,WOL
4,239,Justin,DEF,4.7,1.116106,7.937006,LEI
5,284,Foden,MID,6.6,1.197377,9.682550,MCI
6,164,Calvert-Lewin,FWD,7.4,0.979703,9.864346,EVE
7,252,Alisson,GKP,6.0,1.167673,7.928236,LIV
8,225,Praet,MID,5.5,1.073910,6.822223,LEI
9,469,Podence,MID,5.6,1.137792,6.579201,WOL


In [271]:
##########################################################
### TOP POINTS PLAYERS AND TOP PLAYER SCORE DATAFRAMES ###
###           BASED ON PLAYER SCORE CALC V1            ###
##########################################################


##################################################################
# Creating a data frame of players ordered by their total points #
##################################################################

points_top_players = new_slim_player_df[['cost_in_mil',
                                         'team_abv',
                                         'position_abv',
                                         'web_name',
                                         'id',
                                         'total_points_inc_prv_season',
                                         'gw_multi',
                                         'nxt_3_wk_blended_match_multi',
                                         'player_score_v1'
                                        ]].copy()

points_top_players = points_top_players[points_top_players['player_score_v1'] > 0.0]
points_top_players = points_top_players[points_top_players['nxt_3_wk_blended_match_multi'] > 0.98]
points_top_players = points_top_players.sort_values(by=['total_points_inc_prv_season'], ascending=False).reset_index(drop = True)


###########################################################################################################
# Creating a data frame of players ordered by their player score (to get the most cost effective players) #
###########################################################################################################

player_score_top_players = new_slim_player_df[['cost_in_mil',
                                               'team_abv',
                                               'position_abv',
                                               'web_name',
                                               'id',
                                               'total_points_inc_prv_season',
                                               'gw_multi',
                                               'nxt_3_wk_blended_match_multi',
                                               'player_score_v1'
                                              ]].copy()

player_score_top_players = player_score_top_players[player_score_top_players['player_score_v1'] > 0.0]
player_score_top_players = player_score_top_players[player_score_top_players['nxt_3_wk_blended_match_multi'] > 0.98]
player_score_top_players = player_score_top_players.sort_values(by=['player_score_v1'], ascending=False).reset_index(drop = True)


# print('Table of Top Players Ordered By Total Points:')
# points_top_players.head(20)

# print('Table of Top Players Ordered By Player Score:')
# player_score_top_players.head(20)

################################################
### Optimal team logic Using Player Score V1 ###
################################################

# Function which produces the optimal team based on:
    # A specified number of star players (picking solely based on top points)
    # Filling the rest of the team based on points per million 
    # A set the budget you want to spend on your team
    # Specify the minimum cost you want left to spend on the final players (most likely your subs)

def get_money_team_objects(points_top_players=points_top_players, 
                           player_score_top_players=player_score_top_players,
                           budget = 100, 
                           star_player_limit = 3, 
                           gk = 2, 
                           df = 5, 
                           md = 5, 
                           fwd = 3, 
                           min_player_cost = 4.5):
    
    money_team = pd.DataFrame(columns = ['id', 'web_name', 'position_abv', 'cost_in_mil', 'total_points_inc_prv_season', 'team_abv'])
    star_player_limit = star_player_limit
    budget = budget
    positions = {'GKP': gk, 'DEF': df, 'MID': md, 'FWD': fwd}
    club_player_count = dict((club, 3) for club in list(club_df['short_name'].unique()))
    players_left = 15
    min_player_cost = min_player_cost
    
    for index, player in points_top_players.iterrows():
        if len(money_team) < star_player_limit and budget >= player.cost_in_mil and positions[player.position_abv] > 0  and club_player_count[player.team_abv] > 0 and (budget - player.cost_in_mil) >= ((players_left - 1) * min_player_cost):
            my_dict = {'id': player.id,
                       'web_name': player.web_name, 
                       'position_abv': player.position_abv, 
                       'cost_in_mil': player.cost_in_mil, 
                       'total_points_inc_prv_season': player.total_points_inc_prv_season, 
                       'team_abv': player.team_abv}
            money_team = money_team.append(my_dict, ignore_index=True)
            budget -= player.cost_in_mil
            players_left = players_left - 1 
            positions[player.position_abv] = positions[player.position_abv] -1
            club_player_count[player.team_abv] = club_player_count[player.team_abv] -1
    for index, player in player_score_top_players.iterrows():
        if player.id not in money_team['id'].to_list() and budget >= player.cost_in_mil and positions[player.position_abv] > 0 and club_player_count[player.team_abv] > 0 and (budget - player.cost_in_mil) >= ((players_left - 1) * min_player_cost):
            my_dict2 = {'id':player.id,
                        'web_name': player.web_name, 
                        'position_abv': player.position_abv, 
                        'cost_in_mil': player.cost_in_mil, 
                        'total_points_inc_prv_season': player.total_points_inc_prv_season, 
                        'team_abv': player.team_abv}
            money_team = money_team.append(my_dict2, ignore_index=True) 
            budget -= player.cost_in_mil
            players_left = players_left - 1 
            positions[player.position_abv] = positions[player.position_abv] -1
            club_player_count[player.team_abv] = club_player_count[player.team_abv] -1
    return money_team

optimal_team_df = get_money_team_objects(star_player_limit = 6, budget = 99, min_player_cost = 5)
print('Total Team Points: ' + str(optimal_team_df['total_points_inc_prv_season'].sum()))
print('Total Team Cost: ' + str(optimal_team_df['cost_in_mil'].sum()))

optimal_team_df.head(15)

Total Team Points: 284.42
Total Team Cost: 98.99999999999999


,id,web_name,position_abv,cost_in_mil,total_points_inc_prv_season,team_abv
0,164,Calvert-Lewin,FWD,7.4,34.32,EVE
1,390,Son,MID,9.1,30.45,TOT
2,254,Salah,MID,12.1,29.13,LIV
3,166,Richarlison,FWD,8.0,26.34,EVE
4,251,Mané,MID,11.9,23.82,LIV
5,461,Saïss,DEF,5.1,18.71,WOL
6,156,Keane,DEF,5.0,17.37,EVE
7,239,Justin,DEF,4.7,16.92,LEI
8,123,James,DEF,5.1,17.61,CHE
9,455,Patrício,GKP,5.5,13.03,WOL


In [ ]:
### MY TEAM PICKS DATA ###
### Creating a dataframe for my team picks data and saving as csv ###
### Used to see value of current team and highlight:
                                                    # Players to use as captain / subs   
                                                    # When to use wildcards
                                                    # Weak players to transfer out
                    
# ######################################################################
# ### Create session with login details for urls which require login ###
# ######################################################################

login = ''
password = ''

session = requests.session()

## Create session with login details for urls which require login ##
## Add pasword and login email
payload = {'password': password,
           'login': login,
           'redirect_uri': 'https://fantasy.premierleague.com/a/login',
           'app': 'plfpl-web'}
session.post(login_url, data=payload)    

## Getting my team data from the api url, using session as login details required
my_team_ses = (session.get('https://fantasy.premierleague.com/api/my-team/3366741'))

#############################################
#### my_team_picks_df dataframe creation ####
#############################################

# ## Getting my team data from the api url, using session as login details required
## team 1 (a_c_m)
my_team_ses = (session.get('https://fantasy.premierleague.com/api/my-team/3366741'))

## Other teams
# ## Team 2(riia)
# my_team_ses = (session.get('https://fantasy.premierleague.com/api/my-team/6442816'))
## Team 3 (amao)
# my_team_ses = (session.get('https://fantasy.premierleague.com/api/my-team/6443347'))

## Converting that data to json then dataframe
my_team_json = my_team_ses.json()

my_team_picks_df = pd.DataFrame(my_team_json['picks'])

my_team_picks_df['potential_profit'] = my_team_picks_df.purchase_price - my_team_picks_df.selling_price
my_team_picks_df['cost_in_mil'] = my_team_picks_df.selling_price / 10  

# Pulling through player data from 'full_player_df'
my_team_picks_df = my_team_picks_df.merge(full_player_df[['id',
                                                           'first_name',
                                                           'second_name',
                                                           'web_name',
                                                           'total_points',
                                                           'points_per_mil',
                                                           'position_abv',
                                                           'team_abv',
                                                           'prev_season_points_per_gw',
                                                           'nxt_round_play_chance']], 
                                           how = 'left', left_on = ['element'], right_on = ['id'])

my_team_picks_df['team_and_position'] = my_team_picks_df['team_abv'].astype(str) + '-' + my_team_picks_df['position_abv'].astype(str)
my_team_picks_df['total_points_inc_prv_season'] = my_team_picks_df.total_points + my_team_picks_df.prev_season_points_per_gw
my_team_picks_df['prev_season_points_per_mil'] = my_team_picks_df.prev_season_points_per_gw / my_team_picks_df.cost_in_mil

#### Pulling through Gameweek multipliers ###

# trying to do another merge here but not working
# my_team_picks_df = my_team_picks_df.merge(gameweek_multiplier_df[['gw_multi','nxt_3_wk_blended_match_multi']],
#                                           how = 'left',
#                                           left_on = ['team_and_position'], 
#                                           right_on = ['team_and_position'])


my_team_picks_df['gw_multi'] = my_team_picks_df.team_and_position.map(gameweek_multiplier_df.set_index('team_and_position').gw_multi)
# my_team_picks_df['nxt_5_gw_multi  '] = my_team_picks_df.team_and_position.map(gameweek_multiplier_df.set_index('team_and_position').nxt_5_gw_multi)
# my_team_picks_df['nxt_5_wk_blended_match_multi'] = my_team_picks_df.team_and_position.map(gameweek_multiplier_df.set_index('team_and_position').nxt_5_wk_blended_match_multi)
my_team_picks_df['nxt_3_wk_blended_match_multi'] = my_team_picks_df.team_and_position.map(gameweek_multiplier_df.set_index('team_and_position').nxt_3_wk_blended_match_multi)

my_team_picks_df['player_score'] = ((my_team_picks_df.points_per_mil + my_team_picks_df.prev_season_points_per_mil) * my_team_picks_df.nxt_3_wk_blended_match_multi) * my_team_picks_df.nxt_round_play_chance

print('total nxt_3_wk_blended_match_multi: ' + str(my_team_picks_df['nxt_3_wk_blended_match_multi'].sum()))

my_team_picks_df = my_team_picks_df.sort_values(by=['gw_multi'], ascending=False).reset_index(drop = True)
    
# export_to_csv(my_team_picks_df)  
my_team_picks_df.head(15)

### Random Bits / In Development

In [ ]:
#############################################
### money team logic using try and except ###
#############################################

# def get_money_team_objects(points_top_players=points_top_players, 
#                            budget = 100, 
#                            star_player_limit = 3, 
#                            gk = 2, 
#                            df = 5, 
#                            md = 5, 
#                            fwd = 3, 
#                            min_player_cost = 5.0):
#     money_team = pd.DataFrame(columns = ['web_name', 'position_abv', 'cost_in_mil', 'total_points_inc_prv_season', 'team_abv'])
#     star_player_limit = star_player_limit
#     budget = budget
#     positions = {'GKP': gk, 'DEF': df, 'MID': md, 'FWD': fwd}
#     players_left = 15
#     min_player_cost = min_player_cost
#     for index, player in points_top_players.iterrows():
#         try:
#             count_team = money_team.groupby('team_abv').count().ix[player.team_abv, 0]
#         except:
#             count_team = 0
#         if len(money_team) < star_player_limit and budget >= player.cost_in_mil and positions[player.position_abv] > 0 and count_team < 3 and (budget - player.cost_in_mil) >= ((players_left - 1) * min_player_cost):
#             print('------')
#             print('budget: ' + str(budget))
#             print('players_left: ' + str(players_left))
#             print('min_player_cost: ' + str(min_player_cost))
#             print('budget minus player cost: ' + str((budget - player.cost_in_mil)))
#             print('players left x min player cost: ' + str(players_left * min_player_cost))   
#             my_dict = {'web_name': player.web_name, 
#                        'position_abv': player.position_abv, 
#                        'cost_in_mil': player.cost_in_mil, 
#                        'total_points_inc_prv_season': player.total_points_inc_prv_season, 
#                        'team_abv': player.team_abv}
#             money_team = money_team.append(my_dict, ignore_index=True)
#             budget -= player.cost_in_mil
#             players_left = players_left - 1 
#             positions[player.position_abv] = positions[player.position_abv] -1
#     for index, player in player_score_top_players.iterrows():
#         try:
#             count_team = money_team.groupby('team_abv').count().ix[player.team_abv, 0]
#         except:
#             count_team = 0
#         if player.web_name not in money_team['web_name'].to_list() and budget >= player.cost_in_mil and positions[player.position_abv] > 0 and count_team < 3 and (budget - player.cost_in_mil) >= ((players_left - 1) * min_player_cost):
#             print('------')
#             print('budget: ' + str(budget))
#             print('players_left: ' + str(players_left))
#             print('min_player_cost: ' + str(min_player_cost))
#             print('budget minus player cost: ' + str((budget - player.cost_in_mil)))
#             print('players left x min player cost: ' + str(players_left * min_player_cost)) 
#             my_dict2 = {'web_name': player.web_name, 
#                         'position_abv': player.position_abv, 
#                         'cost_in_mil': player.cost_in_mil, 
#                         'total_points_inc_prv_season': player.total_points_inc_prv_season, 
#                         'team_abv': player.team_abv}
#             money_team = money_team.append(my_dict2, ignore_index=True) 
#             budget -= player.cost_in_mil
#             players_left = players_left - 1 
#             positions[player.position_abv] = positions[player.position_abv] -1
#     return money_team

# money_team = get_money_team_objects()
# money_team.head(15)

In [ ]:
##############################################
####### Show Datatypes Of Dataframe ##########
##############################################

# dataTypeSeries = <insert dataframe name>.dtypes
# print('Data type of each column of Dataframe :')
# print(dataTypeSeries)

In [ ]:
###########################################
### print column names from a dataframe ###
###########################################

# for col in club_df.columns: 
#     print(col) 

In [ ]:
# other things you need login data to see:

print(my_team_json.keys())

my_team_chips_df = pd.DataFrame(my_team_json['chips'])
my_team_chips_df.head()

print(my_team_picks_df.selling_price.sum()+23)


In [ ]:
## LOGIC TO MAKE A PIVOT TABLE FOR AVERAGE COST PER POSITION (ONLY PLAYERS WHO HAVE PLAYED) AND CONVERT TO A DATAFRAME ##
# The plan was to blend this with player value (points per mil) 
# but I realised this would be detrimental to those players who are less than the average value
# so I shelved this logic

# There might be something in belnd if player cost more than average else just use the points per mil, 
# so that more expensive players are not penalised as much for their higher price

# Without logic like this the very top expensive players like Salah would not have made it into my 2020 team 
# and therefore not banked 40 points as my captain in Gameweek 1!

# Adding Average Cost of position and points per average mil cost of position:

    # A cut of the slim player table with only players who have played for more than 0 minutes
player_has_played_df = full_player_df[full_player_df.minutes != 0]
    # Creating a pivot table to calculate the average cost for the position (only using players who have played this season)
average_cost_pivot = player_has_played_df.pivot_table(index=['position_abv'],
                                                      values=['cost_in_mil'], 
                                                      aggfunc='mean')
    # Then converting to a dataframe and renaming 
average_cost_df = pd.DataFrame(average_cost_pivot.to_records()).rename(columns={"cost_in_mil": "avg_position_cost"})